### Use ``Open Route Service API`` to retrieve distance matrix (Single Call version)
#### Quota 500 per day, approx 1 year of merged rental-place dataset per day. 
#### Specify the ``year`` of rental and place dataset before running 
#### The following processes 2022 separately since no place data is available for 2022, by assumption we use the most recent 2021 instead

In [1]:
import pandas as pd
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../scripts/')
import openrouteservice as ors
from add_distance import add_distance_time

In [2]:
PLACE_COLS = ['Place_Names', 'latitude', 'longitude', 'year', 'place_type', 'SA2_CODE']
place_df = pd.read_csv('../../data/curated/features_of_interst/place_all_with_sa2.csv')[PLACE_COLS]
print(place_df.shape)

(70444, 6)


### Alter the place and property dataframes for specific year (2013, 2014, ... 2022)

In [3]:
place_df = place_df[place_df['year'] == 2021]
place_df = place_df.drop_duplicates(subset=['Place_Names', 'latitude']).reset_index(drop=True)
print(place_df.shape)
place_df.head(10)

(8687, 6)


,Place_Names,latitude,longitude,year,place_type,SA2_CODE
0,WARRAGUL HIGH SCHOOL,-38.16707,145.92237,2021,secondary,205011079
1,MERRILANDS COLLEGE (P-12),-37.69667,145.00277,2021,secondary,209021524
2,Oberon High School,-38.23081,144.36536,2021,secondary,203031489
3,WINDSOR TECHNICAL SCHOOL,-37.85679,144.99598,2021,secondary,206061136
4,WODONGA HIGH SCHOOL,-36.12561,146.88105,2021,secondary,204031492
5,OAKLEIGH HIGH SCHOOL,-37.90290,145.11653,2021,secondary,212051326
6,SAINT KILDA ALTERNATIVE HIGH SCHOOL,-37.85818,144.98126,2021,secondary,206051513
7,Castlemaine High School,-37.05054,144.22750,2021,secondary,202021027
8,TEMPLESTOWE HIGH SCHOOL,-37.77124,145.10626,2021,secondary,207021160
9,MELBOURNE COLLEGE OF TEXTILES,-37.73485,144.93681,2021,secondary,206011109


In [4]:
PROPERTY_COLS = ['address', 'latitude', 'longitude', 'nbed', 'nbath', 'ncar', 'weekly_rent', 'type', 'postcode', 'year', 'month', 'residence_type', 'SA2_CODE']
property_df = pd.read_csv('../../data/curated/property_all_with_SA2/2022_property_with_SA2.csv')[PROPERTY_COLS]
# 2022 example
print(property_df.shape)
property_df.head(10)

(73107, 13)


,address,latitude,longitude,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE
0,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027
1,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,2,House,202021027
2,"27 BARKLY STREET, CHILTERN",-36.154880,146.605010,3.0,1.0,2,400,Rental_residential,3683,2022,9,Other,204031070
3,"17 PARK STREET, CHILTERN",-36.148025,146.608721,3.0,2.0,0,350,Rental_residential,3683,2022,8,Other,204031070
4,"1/20 NICKLESS STREET, CHILTERN",-36.154388,146.603242,2.0,1.0,1,320,House,3683,2022,7,House,204031070
5,"1/20 NICKLESS STREET, CHILTERN",-36.154388,146.603242,2.0,1.0,1,320,House,3683,2022,5,House,204031070
6,"1/20 NICKLESS STREET, CHILTERN",-36.154388,146.603242,2.0,1.0,1,300,House,3683,2022,1,House,204031070
7,"40 EPSOM ROAD, CHILTERN",-36.151454,146.605909,3.0,1.0,0,370,House,3683,2022,6,House,204031070
8,"7 CONNESS STREET, CHILTERN",-36.151025,146.607179,2.0,1.0,0,350,House,3683,2022,5,House,204031070
9,"3/1 OXFORD STREET, CHILTERN",-36.150694,146.606488,2.0,1.0,1,300,none,3683,2022,4,Other,204031070


In [5]:
merged_df = property_df.merge(place_df, how='inner', on='SA2_CODE')
print(f"shape = {merged_df.shape}")
print(f"Distinct sa2 codes = {merged_df['SA2_CODE'].nunique()}")
merged_df = merged_df.rename(columns={'latitude_x': 'latitude_ori', 'longitude_x': 'longitude_ori', 'latitude_y': 'latitude_des', 'longitude_y': 'longitude_des', 'year_x': 'year'})
merged_df.drop('year_y', axis=1, inplace=True)
merged_df.head(15)

shape = (1276996, 18)
Distinct sa2 codes = 502


,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,month,residence_type,SA2_CODE,Place_Names,latitude_des,longitude_des,place_type
0,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,Castlemaine High School,-37.05054,144.22750,secondary
1,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,CASTLEMAINE TECHNICAL COLLEGE,-37.06346,144.22098,secondary
2,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,Castlemaine Secondary College- Junior Campus,-37.05054,144.22750,secondary
3,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,Castlemaine Secondary College - Senior Campus,-37.07707,144.21453,secondary
4,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,OLIVET CHRISTIAN COLLEGE,-37.08848,144.20543,secondary
5,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,WINTERS FLAT PRIMARY SCHOOL,-37.07016,144.20749,primary
6,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,CASTLEMAINE NORTH PRIMARY SCHOOL,-37.05783,144.21899,primary
7,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,CASTLEMAINE PRIMARY SCHOOL,-37.06660,144.22233,primary
8,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,CHEWTON PRIMARY SCHOOL,-37.08233,144.26116,primary
9,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,3,House,202021027,CAMPBELLS CREEK PRIMARY SCHOOL,-37.09293,144.20466,primary


### Create the directory to store merged distance data

In [6]:
if not os.path.exists('../../data/featured'):
    os.makedirs('../../data/featured')

#### Specify the ``client`` and ``year`` here, as well as adding a mode of testing/saving

In [7]:
client = ors.Client('5b3ce3597851110001cf6248e33390389e0b4e0292fb3dcd2dec286b') # Phikhocaz's api key 500

In [8]:
added_distance_merged_df = add_distance_time(merged_df, 2022, client, 'saving') # specify the year here, client id, and mode of running (testing or saving)
#added_distance_merged_df = add_distance_time(subset, 2013)

Executing SA2 Code 202021027
Subset size = 1708, Places count = 14, Property count = 122
To Place Distance Grand List, Normal Branch, length = 1708
Executing SA2 Code 204031070
Subset size = 153, Places count = 9, Property count = 17
To Place Distance Grand List, Normal Branch, length = 1861
Executing SA2 Code 210021235
Subset size = 913, Places count = 11, Property count = 83
To Place Distance Grand List, Normal Branch, length = 2774
Executing SA2 Code 211051275
Subset size = 186, Places count = 3, Property count = 62
To Place Distance Grand List, Normal Branch, length = 2960
Executing SA2 Code 211051278
Subset size = 3276, Places count = 18, Property count = 182
To Place Distance Grand List, Normal Branch, length = 6236
Executing SA2 Code 211051276
Subset size = 2046, Places count = 22, Property count = 93
To Place Distance Grand List, Normal Branch, length = 8282
Executing SA2 Code 205041094
Subset size = 3978, Places count = 39, Property count = 102
To Place Distance Grand List, Sl

In [9]:
added_distance_merged_df.head(20)

,address,latitude_ori,longitude_ori,nbed,nbath,ncar,weekly_rent,type,postcode,year,...,longitude_des,place_type,dist_to_place_M,dist_to_place_KM,time_to_place_S,time_to_place_MIN,dist_to_cbd_M,dist_to_cbd_KM,time_to_cbd_S,time_to_cbd_MIN
0,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.22750,secondary,7895.97,7.89597,601.73,10.028833,116076.56,116.07656,5236.62,87.277
1,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.22098,secondary,5254.10,5.25410,441.94,7.365667,116076.56,116.07656,5236.62,87.277
2,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.22750,secondary,7895.97,7.89597,601.73,10.028833,116076.56,116.07656,5236.62,87.277
3,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.21453,secondary,6224.54,6.22454,625.11,10.418500,116076.56,116.07656,5236.62,87.277
4,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.20543,secondary,8087.73,8.08773,622.79,10.379833,116076.56,116.07656,5236.62,87.277
5,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.20749,primary,6406.50,6.40650,619.35,10.322500,116076.56,116.07656,5236.62,87.277
6,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.21899,primary,6107.61,6.10761,423.85,7.064167,116076.56,116.07656,5236.62,87.277
7,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.22233,primary,4592.83,4.59283,320.81,5.346833,116076.56,116.07656,5236.62,87.277
8,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.26116,primary,673.75,0.67375,57.82,0.963667,116076.56,116.07656,5236.62,87.277
9,"231 MAIN ROAD, CHEWTON",-37.080966,144.265555,2.0,1.0,3,380,House,3451,2022,...,144.20466,primary,8474.68,8.47468,646.31,10.771833,116076.56,116.07656,5236.62,87.277
